<a href="https://colab.research.google.com/github/Jason910315/fine-tuning-llm-model/blob/main/fine_tuning_fortune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
# unsloth 是一個用於大型語言模型微調的工具，可以讓模型運行更快，佔用更少內存
!pip install unsloth

# 卸載當前已安装的 unsloth 包（如果已安装），然後從 Github 安裝最新版本。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# bitsandbytes 是一個用於量化和優化模型的工具，幫著減少模型占用的內存
# unsloth_zoo 包含了一些預訓練模型或其他工具。
!pip install bitsandbytes unsloth_zoo

# Load pretrained model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # 模型處理上下文的最大長度
dtype = None           # 資料類型，讓模型選擇最適合的精度
load_in_4bit = True    # 使用 4bits 減少模型精度以節省內存

# 載入預訓練模型，並獲得 tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# Test

In [ ]:
# 定義提示風格
prompt_style = """以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通卜卦、星象和運勢預測的算命大師。
請回答以下算命問題

### 問題：
{}

### 回答：
<think>{}"""
question = "1992年閏年四月初九巳时生人，女，想了解健康運勢"

In [ ]:
# 準備推理模型
FastLanguageModel.for_inference(model)
# 把 question、空字串傳入 template 的空格，並將文字轉成模型可以吃的張量輸入
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 讓模型根據問題生成回答，最多 1200 個詞
outputs = model.generate(
    input_ids=inputs.input_ids,  # 輸入數字序列
    attention_mask=inputs.attention_mask,  # 註意力遮罩，讓模型知道哪部分更重要
    max_new_tokens=1200,
    use_cache=True,  # 使用緩存加速生成
)

# 解碼模型輸出為可讀文本
response = tokenizer.batch_decode(outputs)
print(response[0])

# Load dataset

In [ ]:
# 定義格式化提示的字串，用於微調的模板(提示詞將模型角色的形容更加精準)
train_prompt_style = """以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通八字算命、 紫微斗數、 風水、易經卦象、塔羅牌占卜、星象、面相手相和運勢預測等方面的算命大师。
請回答以下算命問題

### 問題：
{}

### 回答：
<思考>
{}
</思考>
{}"""


In [ ]:
EOS_TOKEN = tokenizer.eos_token

from datasets import load_dataset
# 以前 200 筆數據作為訓練，並使用預設的語言(default)
dataset = load_dataset("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)
print(dataset.column_names)

In [ ]:
# 格式化資料集中的每筆記錄
def formatting_prompts_func(examples):
  # 從資料集提取問題、思考過程、以及回答
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]
  texts = []  # 儲存格式化後的文本，用於輸入到微調
  # 將每個問題、思考過程、回答都嵌入提示詞模板，並輸入到 texts
  for input, cot, output in zip(inputs, cots, outputs):
    # 最後為每個紀錄加上結尾
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)
  # 返回所有格式化文本的字典
  return {
      "text": texts,
  }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]  # 印出第一筆記錄來觀察


# Fine tuning

In [ ]:
FastLanguageModel.for_training(model)

# 使用 Parameter-Efficient Fine Tuning，並將模型轉為準備微調的狀態
model = FastLanguageModel.get_peft_model(
    model,     # 已經加載過的預訓練模型
    r = 16,     # LoRA 的的秩，決定可訓練的參數量
    # 指定模型中需要微調的區塊，包括 Attention 的 query、key、 value、 output
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 設定 LoRA 的超參數
    lora_dropout = 0, # 設定參數丟棄綠
    bias = "none",    # 是否添加偏置項
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from trl import SFTTrainer  # SFFTaininer，用於監督式微調
from transformers import TrainingArguments  # 用於設定訓練參數
from unsloth import is_bfloat16_supported

# 創建一個 SFFTrainer 物件
trainer = SFTTrainer(
    model=model,   # 傳入要微調的模型
    tokenizer=tokenizer,  # 用於處理文本數據
    train_dataset=dataset,  # 訓練資料集
    dataset_text_field="text",  # 指定訓練資料集中的文本標題 (上面定義過的)
    max_seq_length=max_seq_length,
    dataset_num_proc=2,    # 資料處理的並行數
    packing=False,
    # 定義訓練參數
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 每個設備 (如 GPU) 上的批量大小
        warmup_steps=5,  # 預熱，訓練開始時學習綠開始增加的步數
        max_steps=75,  # 最大訓練步數 (epoches)
        learning_rate=2e-4,  # 學習率
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # 每隔多少步記錄一次日誌
        optim="adamw_8bit",  # 使用的優化器，用於調整模型參數
        weight_decay=0.01,  # 權重衰減，防止過擬合
        lr_scheduler_type="linear",  # 控制學習率的變化方式
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:
# 開始 fine-tuning
trainer_stats = trainer.train()

# Test Fine tuning

In [ ]:
# 再次印出問體檢視
print(question)

In [ ]:
# 將模型切換到推理模式，準備回答問題
FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=4000,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0])

# Save model to GGUF

In [ ]:
# GGUF 是一種高效存取 LLM 模型的格式，支持多种量化方法 (4、8、16bit)，可以顯著減小模型的大小，便於儲存
# 例如在 Ollama 上部署時，量化後的模型在資源受限的設備上運行更快。

# 用於存取用戶資料
from google.colab import userdata

# 從 Colab 設定中獲得 huggingface 的密鑰
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 保存為 8bit 量化格式（Q8_0）
# 這種格式文件小且運行快，適合部署到資源受限的設備
if True: model.save_pretrained_gguf("model", tokenizer,)

# 保存為 16bit 量化格式
# 量化精度更高，但文件較大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 保存為 4bit 量化格式（q4_k_m）
# 文件最小，但精度稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Upload to HuggingFace

In [ ]:
# 用於創建新的 model repository
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上創建一個新的倉庫
create_repo("Jasonyang0315/fortunetelling_model", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 將模型和 tokenizer 上傳
model.push_to_hub_gguf("Jasonyang0315/fortunetelling_model", tokenizer, token=HUGGINGFACE_TOKEN)